In [1]:
import os
import pandas as pd
import numpy as np

import matplotlib.pyplot as plot
import math
import seaborn as sns
from scipy.stats import binom

from datetime import timedelta
from datetime import datetime

import requests
import json 

import re

In [2]:
#id directories
data_dir = os.getcwd()

In [3]:
#supply data
items_df = pd.read_excel('item_sets.xlsx')
orders_df = pd.read_excel('Critical_Supply_Dashboard_19 items_Aug_19.xlsx', 
                          sheet_name='DATA_Orders')
orders_df.columns = orders_df.columns.str.replace(' ', '')
items_df.columns = items_df.columns.str.replace(' ', '')
orders_df.columns = orders_df.columns.str.replace('.', '_')
orders_df = orders_df.iloc[1:]
orders_df = orders_df[orders_df['Category'].notnull()]

In [ ]:
#demand data
r = requests.get('https://data.seattle.gov/resource/33kz-ixgy.json')
calls_df = pd.read_json(r.text)

r = requests.get('https://data.seattle.gov/resource/fire-911.json')
fire_reponses_df = pd.read_json(r.text)

call_types_df = pd.read_excel('City_of_Seattle_set_of_call_types.xlsx')

In [ ]:
calls_df['classification'] = ''
calls_df['date_clean'] = ''
calls_df['week'] = ''
calls_df['year'] = ''

for i in range(len(calls_df)):
    classification_temp = re.split(' - ', calls_df['initial_call_type'].iloc[i])
    time_temp = re.split(' ', calls_df['original_time_queued'].iloc[i])
    calls_df['classification'].iloc[i] = classification_temp[0]
    calls_df['date_clean'].iloc[i] = time_temp[0]

for i in range(len(calls_df)):
    date_temp = datetime.strptime(calls_df['date_clean'].iloc[i], "%d/%W/%Y")
    calls_df['year'].iloc[i] = date_temp.isocalendar()[0]
    calls_df['week'].iloc[i] = date_temp.isocalendar()[1]

In [ ]:
#calculate arrival rates per week
calls_df_arrival_per_week = calls_df[['classification',
                                              'year', 
                                              'week', 'cad_event_number']].groupby(['classification',
                                              'year', 
                                              'week']).agg('count')

In [ ]:
#clean orders data frame to get types

orders_df['FullItemList'] = ''
temp = ['Disinfectant Wipes', 
        'Hand Sanitizer','Hand Soap']


for i in range(len(orders_df)):
    
    category_temp = orders_df['Category'].iloc[i]
    
    if category_temp in temp:
        orders_df['FullItemList'].iloc[i] = category_temp
    elif category_temp in ['N-95 Filter']:
        orders_df['FullItemList'].iloc[i] = 'Masks (N95)'   
    elif category_temp in ['Surgical Mask']:
        orders_df['FullItemList'].iloc[i] = 'Masks (Surgical)'
    elif category_temp in ['PAWS']:
        orders_df['FullItemList'].iloc[i] = 'Antimicrobial (PAWS) Wipes'
    elif category_temp in ['DS', 'DSS']:
        orders_df['FullItemList'].iloc[i] = 'Disinfecting Solution'
    elif category_temp in ['IA']:
        orders_df['FullItemList'].iloc[i] = 'Isopropyl Alcohol'
    elif ((category_temp.find('Gloves') != -1) and (orders_df['Description'].iloc[i].find('exam') != -1)):
        orders_df['FullItemList'].iloc[i] = 'Nitrile Gloves (Public Safety)'
    elif ((category_temp.find('Gloves') != -1) and (orders_df['Description'].iloc[i].find('exam') == -1)):
        orders_df['FullItemList'].iloc[i] = 'Nitrile Gloves (General Use)'
    elif category_temp in ['CMask']:
        orders_df['FullItemList'].iloc[i] = 'Masks (Cloth)'
    elif category_temp in ['Therm']:
        orders_df['FullItemList'].iloc[i] = 'Thermometer'
    elif (category_temp.find('Tyvek') != -1):
        if (category_temp.find('NH') != -1):
            orders_df['FullItemList'].iloc[i] = 'Tyvek Suits'
        else:
            orders_df['FullItemList'].iloc[i] = 'Tyvek Suits w/hoods'
    elif category_temp in ['Gowns']:
        orders_df['FullItemList'].iloc[i] = 'Surgical Gowns'
    elif category_temp in ['HS Refill Bags']:
        orders_df['FullItemList'].iloc[i] = 'Purell (1200 ml stand refill)'
    elif (category_temp.find('Toilet') != -1):
        orders_df['FullItemList'].iloc[i] = 'Toilet Paper'
    else:
        orders_df['FullItemList'].iloc[i] = 'NA'
        
orders_df = orders_df[orders_df['FullItemList'] != 'NA']

In [ ]:
#delay time df
delay_time_df = orders_df[['FullItemList', 'Date', \
                           'EstimatedDeliveryDate']]

#clean to only include df with estimated/realized delay times
delay_time_df = delay_time_df[delay_time_df\
                              ['EstimatedDeliveryDate'] != 'TBD']
delay_time_df = delay_time_df[delay_time_df['EstimatedDeliveryDate']\
                              .notnull()]
delay_time_df['Date'] = pd.to_datetime(delay_time_df['Date'])
delay_time_df['EstimatedDeliveryDate'] = \
(delay_time_df['EstimatedDeliveryDate']).astype(str)

#to filter out other misc errors
delay_time_df['filter'] = delay_time_df['EstimatedDeliveryDate']\
.str.contains('-') 
delay_time_df = delay_time_df[delay_time_df['filter'] == True]
delay_time_df['EstimatedDeliveryDate'] = \
pd.to_datetime(delay_time_df['EstimatedDeliveryDate'])

#calculate delay time
delay_time_df['delay_time'] = \
delay_time_df['EstimatedDeliveryDate'] - delay_time_df['Date']

#merge with items df
delay_time_df = delay_time_df.merge(items_df, 
                                    on = 'FullItemList', 
                                    how='left')

#filter negative days
delay_time_df['delay_time_days'] = ''


for i in delay_time_df.index:
    delay_time_df['delay_time_days'].iloc[i] = \
    delay_time_df['delay_time'].iloc[i].days
    
delay_time_df = delay_time_df[delay_time_df['delay_time_days'] > 0]
delay_time_df['delay_time_days'] = \
delay_time_df['delay_time_days'].astype(int)

In [ ]:
max_delay = math.floor(delay_time_df['delay_time_days'].max()/7)
min_delay = math.floor(delay_time_df['delay_time_days'].min()/7)

In [ ]:
#generate f_k_i_tdiff distributions
def lead_dist_gen():
    
    for i in delay_time_df['FullItemList'].unique():
        delay_time_df_dist_temp = \
        delay_time_df[delay_time_df['FullItemList'] == i]
        delay_time_df_dist_temp = \
        delay_time_df_dist_temp[['ItemListID', 
                                 'FullItemList', 
                                 'delay_time_days', 
                                 'filter']]
        
        #count by delay time
        delay_time_df_dist_temp = \
        delay_time_df_dist_temp.groupby(['ItemListID', 
                                          'FullItemList', 
                                          'delay_time_days'])\
        .agg({'filter': 'count'}).reset_index()

        delay_time_df_dist_temp2 = \
        delay_time_df_dist_temp[['ItemListID', 
                                 'filter']]\
        .groupby(['ItemListID'])\
        .agg({'filter': 'sum'})\
        .reset_index()

        delay_time_df_dist_temp = delay_time_df_dist_temp.\
        merge(delay_time_df_dist_temp2, 
              on = 'ItemListID',
              how = 'left')

        delay_time_df_dist_temp['probability'] = \
        delay_time_df_dist_temp['filter_x']/\
        delay_time_df_dist_temp['filter_y']

        delay_time_df_dist_temp = \
        delay_time_df_dist_temp[['ItemListID', 'FullItemList', 
                                 'delay_time_days', 'probability']]

    #for s in set(delay_time_df_dist_temp['FullItemList']):
        temp_df = \
        delay_time_df_dist_temp[delay_time_df_dist_temp['FullItemList'] == i]
        delay_time = temp_df['delay_time_days']
        delay_time = [math.floor(x /7) for x in delay_time]
        probability = temp_df['probability']
        #plot.bar(delay_time, probability, color='green')
        #plot.xlabel("delay time (in weeks)")
        #plot.ylabel("probability")
        #plot.title("Delay Probability Distribution for " + i)
        #plot.xticks(np.arange(min_delay, max_delay, step=10))
        #plot.yticks(np.arange(0, 1.2, step=.2))
        #plot.show()
        
        #binom(delay_time, probability)
        
        binom_df = pd.DataFrame(delay_time, probability)

In [ ]:
i = delay_time_df['FullItemList'].unique()[1]
delay_time_df_dist_temp = \
delay_time_df[delay_time_df['FullItemList'] == i]
delay_time_df_dist_temp = \
delay_time_df_dist_temp[['ItemListID', 
                         'FullItemList', 
                         'delay_time_days', 
                         'filter']]

#count by delay time
delay_time_df_dist_temp = \
delay_time_df_dist_temp.groupby(['ItemListID', 
                                  'FullItemList', 
                                  'delay_time_days'])\
.agg({'filter': 'count'}).reset_index()

delay_time_df_dist_temp2 = \
delay_time_df_dist_temp[['ItemListID', 
                         'filter']]\
.groupby(['ItemListID'])\
.agg({'filter': 'sum'})\
.reset_index()

delay_time_df_dist_temp = delay_time_df_dist_temp.\
merge(delay_time_df_dist_temp2, 
      on = 'ItemListID',
      how = 'left')

delay_time_df_dist_temp['probability'] = \
delay_time_df_dist_temp['filter_x']/\
delay_time_df_dist_temp['filter_y']

delay_time_df_dist_temp = \
delay_time_df_dist_temp[['ItemListID', 'FullItemList', 
                         'delay_time_days', 'probability']]

#for s in set(delay_time_df_dist_temp['FullItemList']):
temp_df = \
delay_time_df_dist_temp[delay_time_df_dist_temp['FullItemList'] == i]
delay_time = temp_df['delay_time_days']
delay_time = [math.floor(x /7) for x in delay_time]
probability = temp_df['probability']
#plot.bar(delay_time, probability, color='green')
#plot.xlabel("delay time (in weeks)")
#plot.ylabel("probability")
#plot.title("Delay Probability Distribution for " + i)
#plot.xticks(np.arange(min_delay, max_delay, step=10))
#plot.yticks(np.arange(0, 1.2, step=.2))
#plot.show()

#binom(delay_time, probability)

d = {'probability': probability, 'delay': delay_time}

binom_df = pd.DataFrame(data = d).groupby(['delay']).sum().reset_index()

In [ ]:
binom_df

In [ ]:
binom.pmf(n = binom_df['delay'], p = binom_df['probability'], k = 1)

In [ ]:
#generate f_k_i_tdiff distributions

lead_time_dict = {}
def lead_dist_gen2():
    for i in delay_time_df['FullItemList'].unique():
        temp = \
        delay_time_df[delay_time_df['FullItemList'] == i]

        data_binom = [math.floor(x /7) \
                      for x in temp['delay_time_days']]

        ax = sns.distplot(data_binom,
                          kde=False)
        
        height = [h.get_height() for h in ax.patches]
        xy = [h.get_xy() for h in ax.patches]
        
        print(height)
        print(xy)
        
        plot.cla()

In [ ]:
#####THIS IS THE GOOD ONE########

temp = \
delay_time_df[delay_time_df['FullItemList'] == delay_time_df['FullItemList'].unique()[1]]

data_binom = [math.floor(x /7) \
              for x in temp['delay_time_days']]

ax = sns.distplot(data_binom,
                  kde=False,
                  hist_kws={"linewidth": 15,'alpha':1})

height = [h.get_height() for h in ax.patches]
xy = [h.get_xy() for h in ax.patches]

density_temp = np.zeros(max_delay - min_delay)

for week in range(min_delay, max_delay):
    #check if in week
    n = 0
    while(n < len(xy)-1):
        if week in range(int(xy[n][0]), int(xy[n+1][0])):
            density_temp[week] = height[n]
        n = n+1
    
    if week in range(int(xy[len(xy)-1][0]), max(data_binom)):
        density_temp[week] = height[len(xy)-1]

In [ ]:
density_temp

In [ ]:
xy[len(xy)-1]

In [ ]:
density_temp[10]

In [ ]:
delay_time_df['FullItemList'].unique()[0]

In [ ]:
density_temp

In [ ]:
density_temp

In [ ]:
range(int(xy[0][0]), int(xy[1][0]))

In [ ]:
delay_time_df[delay_time_df['FullItemList'] == delay_time_df['FullItemList'].unique()[7]]

In [ ]:
temp2 = \
temp1[['ItemListID', 
      'FullItemList', 
      'delay_time_days', 
      'filter']]

In [ ]:
#generate f_k_i_tdiff distributions
def lead_dist_gen():
    
    for i in delay_time_df['FullItemList'].unique():
        delay_time_df_dist_temp = \
        delay_time_df[delay_time_df['FullItemList'] == i]
        delay_time_df_dist_temp = \
        delay_time_df_dist_temp[['ItemListID', 
                                 'FullItemList', 
                                 'delay_time_days', 
                                 'filter']]
        
        #count by delay time
        delay_time_df_dist_temp = \
        delay_time_df_dist_temp.groupby(['ItemListID', 
                                          'FullItemList', 
                                          'delay_time_days'])\
        .agg({'filter': 'count'}).reset_index()
        
        

        delay_time_df_dist_temp2 = \
        delay_time_df_dist_temp[['ItemListID', 
                                 'filter']]\
        .groupby(['ItemListID'])\
        .agg({'filter': 'sum'})\
        .reset_index()

        delay_time_df_dist_temp = delay_time_df_dist_temp.\
        merge(delay_time_df_dist_temp2, 
              on = 'ItemListID',
              how = 'left')
        
        #print(delay_time_df_dist_temp)

        delay_time_df_dist_temp['probability'] = \
        delay_time_df_dist_temp['filter_x']/\
        delay_time_df_dist_temp['filter_y']

        delay_time_df_dist_temp = \
        delay_time_df_dist_temp[['ItemListID', 'FullItemList', 
                                 'delay_time_days', 'probability']]
        
        temp = delay_time_df[delay_time_df['FullItemList'] == 'Disinfectant Wipes']

data_binom = [math.floor(x /7) \
                      for x in temp['delay_time_days']]
ax = sns.distplot(data_binom)#,
                  #kde=False#,
                  #color='skyblue'#,
                  #hist_kws={"linewidth": 5,'alpha':1})#.get_lines().get_data()
    

ax.set(xlabel='Binomial', ylabel='Frequency')

In [ ]:
set(delay_time_df['FullItemList'])

In [ ]:
temp = delay_time_df[delay_time_df['FullItemList'] == 'Disinfectant Wipes']
#data_binom = temp['delay_time_days']

data_binom = [math.floor(x /7) \
                      for x in temp['delay_time_days']]
ax = sns.distplot(data_binom)#,
                  #kde=False#,
                  #color='skyblue'#,
                  #hist_kws={"linewidth": 5,'alpha':1})#.get_lines().get_data()
    

ax.set(xlabel='Binomial', ylabel='Frequency')

In [ ]:
barX = [h.get_height() for h in ax.patches]

In [ ]:
[h.get_xy() for h in ax.patches]

In [ ]:
barX

In [ ]:
i = delay_time_df['FullItemList'].unique()[0]

delay_time_df_dist_temp = \
delay_time_df[delay_time_df['FullItemList'] == i]
delay_time_df_dist_temp = \
delay_time_df_dist_temp[['ItemListID', 
                         'FullItemList', 
                         'delay_time_days', 
                         'filter']]

#count by delay time
delay_time_df_dist_temp = \
delay_time_df_dist_temp.groupby(['ItemListID', 
                                  'FullItemList', 
                                  'delay_time_days'])\
.agg({'filter': 'count'}).reset_index()

delay_time_df_dist_temp2 = \
delay_time_df_dist_temp[['ItemListID', 
                         'filter']]\
.groupby(['ItemListID'])\
.agg({'filter': 'sum'})\
.reset_index()

delay_time_df_dist_temp = delay_time_df_dist_temp.\
merge(delay_time_df_dist_temp2, 
      on = 'ItemListID',
      how = 'left')

delay_time_df_dist_temp['probability'] = \
delay_time_df_dist_temp['filter_x']/\
delay_time_df_dist_temp['filter_y']

delay_time_df_dist_temp = \
delay_time_df_dist_temp[['ItemListID', 'FullItemList', 
                         'delay_time_days', 'probability']]



temp_df = \
delay_time_df_dist_temp[delay_time_df_dist_temp['FullItemList'] == i]
delay_time = temp_df['delay_time_days']
delay_time = [math.floor(x /7) for x in delay_time]
probability = temp_df['probability']
plot.bar(delay_time, probability, color='green')
plot.xlabel("delay time (in weeks)")
plot.ylabel("probability")
plot.title("Delay Probability Distribution for " + s)
plot.xticks(np.arange(min_delay, max_delay, step=10))
plot.yticks(np.arange(0, 1.2, step=.2))
plot.show()
            
            print(binom(delay_time, probability))